# 1. Preprocessing: Merge Clinical + PGS

In [1]:
%load_ext autoreload
%autoreload 2

import os
from tqdm.auto import tqdm
import pathlib

import numpy as np
import pandas as pd
import lifelines
from lifelines import CoxPHFitter
from sklearn.model_selection import StratifiedKFold

from joblib import Parallel, delayed
from tqdm.notebook import tqdm
import neptune
import warnings
warnings.filterwarnings("ignore")
import shutil

dataset_name = "cvd_massive_excl_emb"
path = "/data/analysis/ag-reils/steinfej/code/umbrella/pre/ukbb"
data_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS/data"
dataset_path = f"{data_path}/2_datasets_pre/{dataset_name}"

## Read PRS data

In [2]:
data_pgs = pd.read_feather(f"{dataset_path}/baseline_pgs.feather")
data_pgs_description = pd.read_feather(f"{dataset_path}/baseline_pgs_description.feather")

In [3]:
data_pgs_description

,id,covariate,dtype,isTarget,based_on,aggr_fn
0,1,eid,integer,False,admin,NaN
1,2,PGS000011,numeric,False,PGS,NaN
2,3,PGS000013,numeric,False,PGS,NaN
3,4,PGS000016,numeric,False,PGS,NaN
4,5,PGS000018,numeric,False,PGS,NaN
5,6,PGS000039,numeric,False,PGS,NaN
6,7,PGS000057,numeric,False,PGS,NaN
7,8,PGS000058,numeric,False,PGS,NaN
8,9,PGS000059,numeric,False,PGS,NaN
9,10,PGS000116,numeric,False,PGS,NaN


## Read Clinical data

In [4]:
data_cv = pd.read_feather(f"{dataset_path}/baseline_clinical.feather")
data_cv_description = pd.read_feather(f"{dataset_path}/baseline_clinical_description.feather")

## Merge Genetics + Clinical

In [5]:
data = data_pgs.merge(data_cv, on="eid")
data_description = pd.concat([data_pgs_description, data_cv_description.tail(-1)]).reset_index()
data_description = data_description.drop("index", axis=1)
data_description = data_description.drop("id", axis=1)

In [6]:
data_description.dtypes

covariate     object
dtype         object
isTarget        bool
based_on      object
aggr_fn      float64
dtype: object

In [1]:
covariate_paths = {
    "clinical":("baseline_pgs.feather", "baseline_pgs_description.feather"),
    "pgs":("baseline_clinical.feather", "baseline_clinical_description.feather"),
}

In [5]:
for paths in covariate_paths:
    print(covariate_paths[paths][0])

baseline_pgs.feather
baseline_clinical.feather


## Write to disk

In [7]:
data.to_feather(f"{dataset_path}/baseline.feather")
data_description.to_feather(f"{dataset_path}/baseline_description.feather")